In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## 1. Project Overview:

<font color="blue">
Traffic Sign Classification is very crucial for self-driving cars.
    
In this project, I will use convolutional neural networks that can classify 43 different traffic signs.

<font color="blue">

The dataset consists of 43 different classes of images.

Classes are as listed below:

0 = Speed limit (20km/h)
    
1 = Speed limit (30km/h)
    
2 = Speed limit (50km/h)
    
3 = Speed limit (60km/h)
    
4 = Speed limit (70km/h)
    
5 = Speed limit (80km/h)
    
6 = End of speed limit (80km/h)
    
7 = Speed limit (100km/h)
    
8 = Speed limit (120km/h)
    
9 = No passing
    
10 = No passing for vehicles over 3.5 metric tons
    
11 = Right-of-way at the next intersection
    
12 = Priority road
    
13 = Yield
    
14 = Stop
    
15 = No vehicles
    
16 = Vehicles over 3.5 metric tons prohibited
    
17 = No entry
    
18 = General caution
    
19 = Dangerous curve to the left
    
20 = Dangerous curve to the right
    
21 = Double curve
    
22 = Bumpy road
    
23 = Slippery road
    
24 = Road narrows on the right
    
25 = Road work
    
26 = Traffic signals
    
27 = Pedestrians
    
28 = Children crossing
    
29 = Bicycles crossing
    
30 = Beware of ice/snow
    
31 = Wild animals crossing
    
32 = End of all speed and passing limits
    
33 = Turn right ahead
    
34 = Turn left ahead
    
35 = Ahead only
    
36 = Go straight or right
    
37 = Go straight or left
    
38 = Keep right
    
39 = Keep left
    
40 = Roundabout mandatory
    
41 = End of no passing
    
42 = End of no passing by vehicles over 3.5 metric tons

In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns
import pickle
import random
import cv2

In [ ]:
with open("../input/traffic-signs/train.p", mode='rb') as training_data:
    train = pickle.load(training_data)
with open("../input/traffic-signs/valid.p", mode='rb') as validation_data:
    valid = pickle.load(validation_data)
with open("../input/traffic-signs/test.p", mode='rb') as testing_data:
    test = pickle.load(testing_data)

In [ ]:
X_train, y_train = train["features"], train["labels"]
X_train.shape
#we have 34799 images with 32 x 32 size in 3 color channels.

In [ ]:
y_train.shape

In [ ]:
X_test,y_test = test["features"], test["labels"]
print(X_test.shape)
print(y_test.shape)

In [ ]:
X_valid, y_valid = valid["features"], valid["labels"]
print(X_valid.shape)
print(y_valid.shape)

## 2. Data Visualization

In [ ]:
i = random.randint(0,len(X_train))
plt.imshow(X_train[i])
y_train[i]

In [ ]:
#Lets visualize multiple images:
fig,axes = plt.subplots(5,5,figsize=(20,15))
axes = axes.ravel() # flatten 5x5 matrix into 25 array
for i in range(0,25):
    index = random.randint(1,len(X_train))
    axes[i].imshow(X_train[index])
    axes[i].set_title(y_train[index], fontsize = 15)
    axes[i].axis("off")
plt.subplots_adjust(hspace=0.4)


## 3. Data Preparation and Analysis:

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train,y_train)
# I do now want that neural networks a meaning from the sequence of the data.

In [ ]:
# Lets normalize the images as follows:
X_train = X_train/255
X_test = X_test/255
X_valid = X_valid/255

In [ ]:
#Lets visualize the images after normalization:
fig,axes = plt.subplots(4,4,figsize=(20,15))
axes = axes.ravel() # flatten 5x5 matrix into 25 array
for i in range(0,16):
    index = random.randint(1,len(X_train))
    axes[i].imshow(X_train[index])
    axes[i].set_title(y_train[index], fontsize = 15)
    axes[i].axis("off")
plt.subplots_adjust(hspace=0.4)

## 4. Training Neural Networks:

In [ ]:
plt.figure(figsize=(20,15))
plt.imshow(plt.imread("../input/convolution/conv.png"))
#How neural networks work:

In [ ]:
#> we will also add dropout layer in order to avoid from overfitting
plt.figure(figsize=(20,15))
plt.imshow(plt.imread("../input/dropout/dropout.jpg"))
#Dropout layer drops some of the perceptrons. How dropout layer work:

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, MaxPool2D, Flatten, Dense, Dropout
cnn = Sequential()
cnn.add(Conv2D(filters=16,  kernel_size=3, activation="relu",input_shape=(32,32,3)))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(filters=64, activation="relu",kernel_size=3))
cnn.add(MaxPool2D(pool_size=2, strides=2))
cnn.add(Flatten())
cnn.add(Dense(units=512,activation="relu"))
#cnn.add(Dense(units=80,activation="relu"))
cnn.add(Dropout(0.2))
cnn.add(Dense(units=43,activation="softmax"))
cnn.summary()
        


In [ ]:
cnn.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
history = cnn.fit(X_train,y_train, validation_data=(X_valid,y_valid),epochs=10, batch_size=500)

In [ ]:
cnn.evaluate(X_test,y_test)

## 5. Model Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
predictions = cnn.predict_classes(X_test)
print(classification_report(y_test,predictions))

In [ ]:
cm = confusion_matrix(y_test,predictions)
plt.figure(figsize=(20,20))
sns.heatmap(cm,annot=True, cmap="magma")

In [ ]:
#Lets compare our predictions with the true values in visualization:
#Lets visualize multiple images:
fig,axes = plt.subplots(5,5,figsize=(20,15))
axes = axes.ravel() # flatten 5x5 matrix into 25 array
for i in range(0,25):
    axes[i].imshow(X_test[i])
    axes[i].set_title(f"Prediction = {predictions[i]}\n True Value = {y_test[i]}")
    axes[i].axis("off")
plt.subplots_adjust(hspace=1)